<a href="https://colab.research.google.com/github/m-antonio22069/avaliacao_de_credito/blob/master/Avaliaca0_de_Credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.tree import DecisionTreeClassifier

In [0]:
# Lendo Base de dados
dataframe = pd.read_csv('Credit.csv')

In [0]:
dataframe.head()

In [0]:
# Tratando os dados

lb = LabelEncoder()
labels = ['checking_status','purpose', 'other_parties', 'property_magnitude',
          'own_telephone','foreign_worker', 'housing','personal_status', 'job' ]
for col in labels:
  dataframe[col] = lb.fit_transform(dataframe[col])


dataframe['credit_history'] = dataframe['credit_history'].map({"'critical/other existing credit'":0,
                                                               "'delayed previously'":1,
                                                               "'existing paid'":2,
                                                               "'no credits/all paid'":3,
                                                               "'all paid'":4})

dataframe['savings_status'] = dataframe['savings_status'].map({"'no known savings'":0,
                                                               '<100':1,
                                                               '100<=X<500':2,
                                                               '500<=X<1000':3,
                                                               '>=1000':4})

dataframe['employment'] = dataframe['employment'].map({'unemployed':0,
                                                       '<1':1,
                                                       '1<=X<4':2,
                                                       '4<=X<7':3,
                                                       '>=7':4})

dataframe['other_payment_plans'] = dataframe['other_payment_plans'].map({'bank': 2, 'none': 0, 'stores': 1})

dataframe['class'] = dataframe['class'].map({'good':1, 'bad':0})

In [0]:
# Verificando correlação 
correlacao = dataframe.corrwith(dataframe['class']).iloc[:-1].to_frame()
correlacao['abs']=correlacao[0].abs()
correlacao_ordenada = correlacao.sort_values('abs',ascending=False)[0]

fig, ax = plt.subplots(figsize=(10,20))
sns.heatmap(correlacao_ordenada.to_frame(),cmap='coolwarm',annot=True, 
            vmin=1, vmax=1, ax=ax);

In [0]:
# Removendo colunas com baixas correlações
lista_chaves = list(correlacao_ordenada.keys())
drop_corr = dict(correlacao_ordenada[lista_chaves.index('existing_credits'):])

for i in drop_corr:
    if i in dataframe.columns:
        dataframe = dataframe.drop(columns=[i])

In [0]:
variaveis = dataframe.iloc[:,:11]
classe = dataframe.iloc[:,11]

x_treino, x_teste, y_treino, y_teste = train_test_split(variaveis,
                                                        classe,
                                                        test_size=0.3,
                                                        random_state=0)



In [0]:
# Criando piperlines para otimização dos testes

from sklearn.pipeline import Pipeline

pip_std_Gaussian = Pipeline([('scaler',StandardScaler()),
                                 ('RandomForestRegressor', GaussianNB())])

pip_min_max_Gaussian = Pipeline([('minmax',MinMaxScaler()),
                                 ('RandomForestRegressor', GaussianNB())])


pip_std_tree = Pipeline([('scaler',StandardScaler()),
                         ('DecisionTree', DecisionTreeClassifier(criterion='gini'))])

pip_min_max_tree = Pipeline([('minmax',MinMaxScaler()),
                             ('DecisionTree', DecisionTreeClassifier(criterion='gini'))])

In [171]:
# treinando modelos e retirando seus scores
pip_std_Gaussian.fit(x_treino,y_treino)

pip_min_max_Gaussian.fit(x_treino,y_treino)

Pipeline(memory=None,
         steps=[('minmax', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('RandomForestRegressor',
                 GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)

In [172]:
print("pip_std_Gaussian:", pip_std_Gaussian.score(x_teste,y_teste))
print("pip_min_max_Gaussian:", pip_min_max_Gaussian.score(x_teste,y_teste))

pip_std_Gaussian: 0.7366666666666667
pip_min_max_Gaussian: 0.7366666666666667


In [173]:
pip_std_tree.fit(x_treino,y_treino)

pip_min_max_tree.fit(x_treino,y_treino)

Pipeline(memory=None,
         steps=[('minmax', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('DecisionTree',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='best'))],
         verbose=False)

In [174]:
print("pip_std_tree:", pip_std_tree.score(x_teste,y_teste))
print("pip_min_max_tree:", pip_min_max_tree.score(x_teste,y_teste))

pip_std_tree: 0.6533333333333333
pip_min_max_tree: 0.6666666666666666
